In [5]:
from influxdb_client import InfluxDBClient
import json
import pandas as pd
from datetime import datetime,timedelta

# ''' M1 server influxdb credentials'''
# org = "iit_bh"
# token = "pg0rBVHySellktOpSG2Ee103-wIvSOUBM_s0KXmR0mDWM60Kn0duN0jm6EudRIJ7ogpNs1JoAiEqtzcvGca70w=="
# url = "http://10.10.0.175:8086"
# bucket = "iba_data_16mm"

org = "iit_bh"
token = "GEfq64vm4n-7tQ7F8AoGf5OfKlJ41hEF84NdUZ-daQJ_lIBFCzqrFMnIMCL57Dm4GGTCFyUJQsxkTeypWfq0EQ=="
url = "http://127.0.0.1:8086"
bucket = "iba_data_16mm"

# client = InfluxDBClient(url=url, token=token, timeout=25000)
client = InfluxDBClient(url=url, token=token)
query_api = client.query_api()

In [6]:
# region_list = ['CVR_L1', 'CVR_L2', 'CVAH_L1', 'CVAH_L2', 'PinchRoll_L1', 'PinchRoll_L2',\
#                 'SH1', 'Stand_1-6', 'Stand_7-12', 'Stand_13-18', 'Furnace_Exit']
region_list = ['CVR_L1', 'CVR_L2', 'CVAH_L1', 'CVAH_L2', 'PinchRoll_L1', 'PinchRoll_L2',\
                'HMD', 'SH1', 'SH2', 'SH3', 'Stand_1-6', 'Stand_7-12', 'Stand_13-18', 'Furnace_Exit', 'Overall']

In [7]:
summary_dict = {}
top4_anomalous_signals = list()

for region in region_list:
  print("****************** "+region+" ****************")
  measurement = region+"_anomaly_score_cobble"
  start_time = "2024-08-01T13:11:00.000Z"               #hardcoded for now
  end_time = "2024-08-01T13:17:00.000Z"                 #hardcoded for now
  field = "predicted_cobble"
  
  ''' querying influxdb table : <region>_anomaly_score_cobble to fetch timestamps when the predicted cobble == 1'''
  list_of_predicted_cobble_timestamps = list()
  query_for_anomaly_time = 'from(bucket: "'+bucket+'")\
    |> range(start: '+start_time+', stop: '+end_time+' )\
    |> filter(fn: (r) => r["_measurement"] == "'+measurement+'")\
    |> filter(fn: (r) => r["_field"] == "predicted_cobble")\
    |> filter(fn: (r) => r["_value"] == 1)'
    # |> keep(columns: ["_time"])'
  try:
      result = query_api.query(org=org,query=query_for_anomaly_time)
  except Exception as ex:
        print(ex)
  else:
    print(result)
    for table in result:
      for record in table.records:
        # row = ''.join((str(record.get_time()),", ", record.get_field(),", ",str(record.get_value()),"\n"))
        list_of_predicted_cobble_timestamps.append(record.get_time())
  # print(list_of_predicted_cobble_timestamps)
  # print("\n\n")

  # querying influxdb table : <region>_loss_per_signal to fetch value of signals at given timestamp
  measurement = region+"_loss_per_signal"
  
  for timestamp in list_of_predicted_cobble_timestamps:
    timestamp = timestamp.strftime('%Y-%m-%dT%H:%M:%S.000Z')

    if timestamp not in summary_dict:
      summary_dict[timestamp] = {}
    summary_dict[timestamp][region] = []
    
    query_for_loss_signals = 'from(bucket: "'+bucket+'")\
      |> range(start: '+start_time+', stop: '+end_time+' )\
      |> filter(fn: (r) => r["_measurement"] == "'+measurement+'")\
      |> filter(fn: (r) => r["_time"] == '+timestamp+')\
      |> group()\
      |> sort(columns: ["_value"], desc: true)\
      |> limit(n:4)\
      |> keep(columns: ["_field"])'

    try:
      result_for_loss_signals = query_api.query(org=org,query=query_for_loss_signals)
    except Exception as ex:
        print("!!!!!!!! ERROR : "+ex)
    else:
      top4_anomalous_signals.clear()
      for table in result_for_loss_signals:
        for record in table.records:
            top4_anomalous_signals.append(record.get_field())
      summary_dict[timestamp][region].extend(top4_anomalous_signals)


****************** CVR_L1 ****************
<urllib3.connection.HTTPConnection object at 0x00000153B9551510>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it
****************** CVR_L2 ****************
<urllib3.connection.HTTPConnection object at 0x00000153B95523B0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it
****************** CVAH_L1 ****************
<urllib3.connection.HTTPConnection object at 0x00000153B9552680>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it
****************** CVAH_L2 ****************
<urllib3.connection.HTTPConnection object at 0x00000153B9552980>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it
****************** PinchRoll_L1 **********

In [37]:
with open('alert_summary_16mm.json', 'w', encoding='utf-8') as f:
    json.dump(summary_dict, f, ensure_ascii=False, indent=4)

In [8]:
summary_to_api = {}

overall_list = []
for key1,val1 in summary_dict.items():
    snapshot_dict = {}
    region_list = []
    # region_dict = {}
    for key2,val2 in val1.items():
        region_dict = {}
        region_dict["region"] = key2
        region_dict["signals"] = val2
        # print(region_dict)
        region_list.append(region_dict)

    key1 = datetime.strptime(key1,'%Y-%m-%dT%H:%M:%S.000Z')
    timestamp_offset = timedelta(hours=5, minutes=30)
    ist_timestamp = key1 + timestamp_offset
    timestamp_formatted = ist_timestamp.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    snapshot_dict["time"] = timestamp_formatted
    snapshot_dict["data"] = region_list
    overall_list.append(snapshot_dict)

summary_to_api[bucket] = overall_list


In [9]:
with open('alert_summary_16mm_new.json', 'w', encoding='utf-8') as f:
    json.dump(summary_to_api, f, ensure_ascii=False, indent=4)